**Train-test split in sequential data**

In [ ]:
"""

In many machine learning applications, one randomly splits the data into training and testing sets. However, with sequential data, there are better approaches.
If we split the data randomly, we risk creating a look-ahead bias, where the model has information about the future when making forecasts.
In practice, we won't have information about the future when making predictions, so our test set should reflect this reality.
To avoid the look-ahead bias, we should split the data by time. We will train on the first three years of data, and test on the fourth year

"""

**Creating Sequences**

In [ ]:
"""

To feed the training data to the model, we need to chunk it first to create sequences that the model can use as training examples.
First, we need to select the sequence length, which is the number of data points in one training example. Let's make each forecast based on the previous 24 hours.
Because data is at 15 minute intervals, we need to use 24 times 4 which is 96 data points.
In each example, the data point right after the input sequence will be the target to predict

"""

**Generating Sequences**

In [ ]:
"""

Iterate over the range of the number of data points minus the length of an input sequence.

Define the inputs x as the slice of df from the ith row to the i + seq_lengthth row and the column at index 1.

Define the target y as the slice of df at row index i + seq_length and the column at index 1.

"""

import numpy as np

def create_sequences(df, seq_length):   ### The loop goes from the first row (i = 0) to the last row where a full sequence can be made
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df) - seq_length):
      	# Define inputs
        x = df.iloc[i : (i + seq_length), 1]
        # Define target
        y = df.iloc[(i + seq_length), 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

"""

Index	      Timestamp	            Consumption
0	      2011-01-01 00:15:00	      -0.70
1	      2011-01-01 00:30:00	      -0.70
2	      2011-01-01 00:45:00	      -0.68
3	      2011-01-01 01:00:00	      -0.66
4	      2011-01-01 01:15:00     	-0.65
5	      2011-01-01 01:30:00     	-0.63

Sequence Length (seq_length): 3


Iteration 1 (i = 0):
---------------------
Sequence (x):
------------
df.iloc[0:3, 1]  # Rows 0, 1, 2 → [-0.70, -0.70, -0.68]
Sequence: [-0.70, -0.70, -0.68]


Target (y):
-----------
df.iloc[3, 1]  # Row 3 → -0.66
Target: -0.66

"""

**Sequential Dataset**

In [ ]:
"""

Just like tabular and image data, sequential data is easiest passed to a model through a torch Dataset and DataLoader.
To build a sequential Dataset, you will call create_sequences() to get the NumPy arrays with inputs and targets, and inspect their shape.
Next, you will pass them to a TensorDataset to create a proper torch Dataset, and inspect its length.

Your implementation of create_sequences() and a DataFrame with the training data called train_data are available

"""






"""

Call create_sequences(), passing it the training DataFrame and a sequence length of 24*4, assigning the result to X_train, y_train.

Define "dataset_train" by calling TensorDataset and passing it two arguments, the inputs and the targets created by create_sequences(), both converted from NumPy arrays to tensors of floats.

"""


import torch
from torch.utils.data import TensorDataset

# Use create_sequences to create inputs and targets
X_train, y_train = create_sequences(train_data , seq_length = 96)
print(X_train.shape, y_train.shape)

# Create TensorDataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))



##### The TensorDataset we have just built behaves the same way as other Torch Datasets we have used before, such us our custom WaterDataset or the ImageFolder dataset; we can pass it to a DataLoader in the same way.

**Recurrent Neural Networks**

In [ ]:
"""

Define the RNN layer passing it the correct values for input_size, hidden_size, num_layers, and batch_first, and assign it to self.rnn

Initialize the first hidden state h0 as a tensor of zeros of the appropriate shape.

Pass the input x and the first hidden state h0 through recurrent layer.

Pass recurrent layer's last output through the linear layer

"""
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size = 1,
            hidden_size = 32,
            num_layers = 2,
            batch_first = True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)
        # Pass x and h0 through recurrent layer
        out, _ = self.rnn(x , h0)
        # Pass recurrent layer's last output through linear layer
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
In our dataset model learns to predict the sum of the last two numbers in a sequence.

For example:

Input: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Output: 19 (because 9 + 10 = 19)


## (1). Generating the Data ; We create sequences of length 10, where the model should predict the sum of the last two values.


import torch
import torch.nn as nn

# Toy dataset: Sequence where the target is the sum of last two numbers
def generate_sequences(num_samples=4, seq_length=10):
    X = []  # Input sequences
    Y = []  # Target values

    for _ in range(num_samples):
        seq = torch.randint(1, 20, (seq_length,)).float()  # Random sequence of numbers
        X.append(seq.unsqueeze(1))  # Make it shape (seq_length, 1)
        Y.append(seq[-2:].sum().unsqueeze(0))  # Target = Sum of last two numbers

    return torch.stack(X), torch.stack(Y)

# Generate 4 sequences
X_train, Y_train = generate_sequences(num_samples=4, seq_length=10)

print("Input X (Sequences):")
print(X_train)  # Shape (4, 10, 1) ; (batch_size, sequence_length, input_size(Each timestep has only one feature) )
print("Target Y (Sum of last two numbers):")
print(Y_train)  # Shape (4, 1)




## (2). Running the Model with This Data


# Define the RNN Model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=32, num_layers=2, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)  # (num_layers=2, batch_size, hidden_size=32)
        out, _ = self.rnn(x, h0)  # Forward pass through RNN
        out = self.fc(out[:, -1, :])  # Take last timestep output
        return out

# Create model
model = Net()

# Forward pass with real data
output = model(X_train)

print("\nModel Predictions:")
print(output)



## (3). Example Output

"""

Input X (Sequences):
tensor([[[ 8.], [10.], [19.], [10.], [ 3.], [ 1.], [12.], [ 4.], [ 3.], [ 5.]],
        [[18.], [ 2.], [ 9.], [ 5.], [ 4.], [ 7.], [14.], [19.], [14.], [ 5.]],
        [[10.], [ 6.], [15.], [17.], [14.], [ 3.], [18.], [19.], [19.], [ 2.]],
        [[ 2.], [12.], [10.], [ 3.], [10.], [19.], [ 3.], [ 2.], [11.], [ 4.]]])

Target Y (Sum of last two numbers):
tensor([[ 8.],  # 3+5
        [19.],  # 14+5
        [21.],  # 19+2
        [15.]]) # 11+4

Model Predictions:
tensor([[10.5732],
        [15.0432],
        [17.2898],
        [12.3023]])
"""

# LSTM and GRU Cell

In [ ]:
"""

In the .__init__() method, define an LSTM layer and assign it to self.lstm.
In the forward() method, initialize the first long-term memory hidden state c0 with zeros.
In the forward() method, pass all three inputs to the LSTM layer: the current time step's inputs, and a tuple containing the two hidden states

"""

class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define lstm layer
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        # Initialize long-term memory
        c0 = torch.zeros(2, x.size(0), 32)
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x , (h0 , c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
"""

Update the RNN model definition in order to obtain a GRU network; assign the GRU layer to self.gru.

"""

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.GRU(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Training and Evaluating RNNs

**Expanding tensors**

In [ ]:
"""

All recurrent layers, RNNs, LSTMs, and GRUs, expect input in the shape: batch size, sequence length, number of features.

But as we loop over the DataLoader, we can see that we got the shape batch size of 32 by the sequence length of 96.
Since we are dealing with only one feature, the electricity consumption, the last dimension is dropped.
We can add it, or expand the tensor, by calling view on the sequence and passing the desired shape.



for seqs, labels in dataloader_train:
      print(seqs.shape) ### Output : torch.Size([32, 96])

seqs = seqs.view(32, 96, 1)
print(seqs.shape)   ### Output : torch.Size([32, 96, 1])
"""
"""

**Squeezing Tensors**

In [ ]:
"""

Conversely, as we evaluate the model, we will need to revert the expansion we have applied to the model inputs which can be achieved through squeezing.
As we iterate through test data batches, we get labels in shape batch size. Model outputs, however, are of shape batch size by 1, our number of features.

We will be passing the labels and the model outputs to the loss function, and each PyTorch loss requires its inputs to be of the same shape.
To achieve that, we can apply the squeeze method to the model outputs. This will reshape them to match the labels' shape

"""

**Training Loop**

In [ ]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(
    net.parameters(), lr=0.001
)
for epoch in range(num_epochs):
    for seqs, labels in dataloader_train:
        seqs = seqs.view(32, 96, 1)
        outputs = net(seqs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

**Evaluation Loop**

In [ ]:
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in test_loader:
        seqs = seqs.view(32, 96, 1)
        outputs = net(seqs).squeeze()
        mse(outputs, labels)

print(f"Test MSE: {mse.compute()}")

In [ ]:
"""

Set up the Mean Squared Error loss and assign it to criterion.
Reshape seqs to (batch size, sequence length, num features), which in our case is (32, 96, 1), and re-assign the result to seqs.
Pass seqs to the model to get its outputs

"""

net = Net()
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
    for seqs, labels in dataloader_train:
        # Reshape model inputs
        seqs = seqs.view(32, 96, 1)
        # Get model outputs
        outputs = net(seqs)
        # Compute loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
"""

Define the Mean Squared Error metrics and assign it to mse.
Pass the input sequence to net, and squeeze the result before you assign it to outputs.
Compute the final value of the test metric assigning it to test_mse.

"""


# Define MSE metric
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(32, 96, 1)
        # Pass seqs to net and squeeze the result
        outputs = net(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")
